In [206]:
import pandas
import math 

In [207]:
match = pandas.read_csv('data/clean/match.csv',sep=',')
match

,id,gameVersion,gameDurationCluster,afkEarlySurrender,tierId,divisionId
0,0,13.19,1,False,4,4
1,1,13.19,2,False,4,4
2,2,13.13,1,False,4,4
3,3,13.13,2,False,4,4
4,4,13.19,1,False,4,4
...,...,...,...,...,...,...
174095,174280,13.14,1,False,1,4
174096,174281,13.13,2,False,1,4
174097,174282,13.14,1,False,1,4
174098,174283,13.13,4,False,1,4


In [208]:
match['tierId']=match['tierId'].astype('float64')
match['tierId']=match['tierId']-1

In [209]:
match['divisionId']=match['divisionId'].astype('float64')
match['divisionId']=match['divisionId']-1

In [210]:
match['level']=match['tierId']+match['divisionId']

In [211]:
match['level'].value_counts().sort_values(ascending=False)

level
3.0    44207
4.0    34424
2.0    30906
5.0    22973
1.0    18588
6.0    15630
0.0     7372
Name: count, dtype: int64

In [212]:
match['level']=match['level'].apply(lambda x: math.log1p(x))
match['level']=match['level'].apply(lambda x: round(x,4))

In [213]:
#drop tierId and divisionId
match=match.drop(['tierId','divisionId'],axis=1)

In [214]:
match['afkEarlySurrender'].replace({True:1,False:0},inplace=True)

In [215]:
match[match['afkEarlySurrender']==1]['afkEarlySurrender'].value_counts()

afkEarlySurrender
1    4225
Name: count, dtype: int64

In [216]:
match[match['afkEarlySurrender']==0]['afkEarlySurrender'].value_counts()

afkEarlySurrender
0    169875
Name: count, dtype: int64

In [217]:
match=match[match['afkEarlySurrender']==0]
match=match.drop(['afkEarlySurrender'],axis=1)

In [218]:
versions=match['gameVersion'].value_counts().sort_values(ascending=False).head(5)
versions

gameVersion
13.20    60325
13.19    31727
13.18    21635
13.17    16873
13.16    12213
Name: count, dtype: int64

tenemos demasiadas versiones asique solo vamos a quedarnos con las 5 mas jugadas

In [219]:
#get versions
versions_ls=versions.index.tolist()
versions_ls

[13.2, 13.19, 13.18, 13.17, 13.16]

In [220]:
match=match[match['gameVersion'].isin(versions_ls)]

In [221]:
match

,id,gameVersion,gameDurationCluster,level
0,0,13.19,1,1.9459
1,1,13.19,2,1.9459
4,4,13.19,1,1.9459
6,6,13.19,4,1.9459
8,8,13.19,1,1.9459
...,...,...,...,...
174083,174268,13.19,3,1.3863
174084,174269,13.17,1,1.3863
174087,174272,13.19,2,1.3863
174088,174273,13.20,2,1.3863


In [222]:
df_dummies_versions=match[['gameVersion']]
df_dummies_versions.loc[:, 'gameVersion']=df_dummies_versions['gameVersion'].astype('category')
df_dummies_versions=pandas.get_dummies(df_dummies_versions)

In [223]:
df_dummies_versions

,gameVersion_13.16,gameVersion_13.17,gameVersion_13.18,gameVersion_13.19,gameVersion_13.2
0,False,False,False,True,False
1,False,False,False,True,False
4,False,False,False,True,False
6,False,False,False,True,False
8,False,False,False,True,False
...,...,...,...,...,...
174083,False,False,False,True,False
174084,False,True,False,False,False
174087,False,False,False,True,False
174088,False,False,False,False,True


In [224]:
match=match.drop(['gameVersion'],axis=1)
match=pandas.concat([match,df_dummies_versions],axis=1)

In [225]:
match

,id,gameDurationCluster,level,gameVersion_13.16,gameVersion_13.17,gameVersion_13.18,gameVersion_13.19,gameVersion_13.2
0,0,1,1.9459,False,False,False,True,False
1,1,2,1.9459,False,False,False,True,False
4,4,1,1.9459,False,False,False,True,False
6,6,4,1.9459,False,False,False,True,False
8,8,1,1.9459,False,False,False,True,False
...,...,...,...,...,...,...,...,...
174083,174268,3,1.3863,False,False,False,True,False
174084,174269,1,1.3863,False,True,False,False,False
174087,174272,2,1.3863,False,False,False,True,False
174088,174273,2,1.3863,False,False,False,False,True


In [226]:
match.to_csv('data/transform/match.csv',index=False,sep=';')